In [13]:
import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd
from shapely import wkb
from difflib import get_close_matches
import dask.dataframe as dd
import dask_geopandas as dgpd
import geopandas as gpd
from shapely.ops import unary_union

import pyarrow.parquet as pq
import pandas as pd
from shapely import wkb
from shapely.ops import unary_union

from cloudpathlib import S3Path
import s3fs
import fsspec
import traceback

import os

# Paths Configuration
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data" #
INPUT_PATH = os.path.join(MAIN_PATH, "input")
BUILDINGS_PATH = os.path.join(INPUT_PATH, "buildings")
ROADS_PATH = os.path.join(INPUT_PATH, "roads")
INTERSECTIONS_PATH = os.path.join(INPUT_PATH, "intersections")
URBAN_EXTENTS_PATH = os.path.join(INPUT_PATH, "urban_extents")
OUTPUT_PATH = os.path.join(MAIN_PATH, "output")
OUTPUT_PATH_CSV = os.path.join(OUTPUT_PATH, "csv")
SEARCH_BUFFER_PATH = os.path.join(INPUT_PATH, "city_info", "search_buffers")
GRIDS_PATH = os.path.join(INPUT_PATH, "city_info", "grids")

fs = s3fs.S3FileSystem(anon=False)


In [2]:


gdf_all_cities = gpd.read_parquet('all_cities_combined.geoparquet')

city_names_500 = pd.read_csv('500cities.csv',sep=';', encoding="ISO-8859-1")

city_names_500["city_name_no_country"] = city_names_500["City Name"].apply(lambda x: x.split(',')[0].replace(" ", "_"))
city_names_500["city_name_no_country"] = city_names_500["city_name_no_country"].apply(lambda x: x.replace(" ", "_"))


city_set = set(city_names_500["city_name_no_country"])


In [3]:


from difflib import get_close_matches
import re

# Function to clean and normalize city names
def normalize_city_name(city):
    """Removes underscores intelligently based on context."""
    return re.sub(r'_(?=[a-z])', '', city)  # Remove underscore only if followed by lowercase letter

# Function to match city names based on refined rules
def refined_fuzzy_match(city, city_set):
    """Matches city names by allowing underscore corrections but avoiding over-relaxed matches."""
    if city in city_set:  # Exact match
        return city
    
    # Normalize city name: remove underscores only where appropriate
    cleaned_city = normalize_city_name(city)
    
    # Find close matches allowing a max difference of 1 edit
    possible_matches = get_close_matches(cleaned_city, city_set, n=1, cutoff=0.9)  # More restrictive cutoff
    return possible_matches[0] if possible_matches else None  # Return matched city or None

# Apply refined fuzzy matching
gdf_all_cities["city_name_matched"] = gdf_all_cities["city_name"].apply(lambda x: refined_fuzzy_match(x, city_set))

# Count how many cities were successfully matched
num_matched = gdf_all_cities["city_name_matched"].notna().sum()
gdf_all_cities['matched_bool'] = gdf_all_cities["city_name_matched"].notna()
gdf_all_cities[gdf_all_cities['matched_bool']][['city_name_matched','merged_geometry']].to_file('../421_cities.shp')
print(num_matched, "cities matched")


selected_cities = gdf_all_cities[gdf_all_cities['matched_bool']][['city_name_matched','merged_geometry']].set_index('city_name_matched')



421 cities matched


/tmp/ipykernel_1369/4097487225.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_all_cities[gdf_all_cities['matched_bool']][['city_name_matched','merged_geometry']].to_file('../421_cities.shp')
/opt/coiled/env/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/opt/coiled/env/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'city_name_matched' to 'city_name_'
  ogr_write(


In [2]:
def s3_save(file, output_file, output_temp_path, remote_path):

    os.makedirs(output_temp_path, exist_ok=True)

    local_temporary_file = f"{output_temp_path}/{output_file}"


    # Save the file based on its extension
    if output_file.endswith(".gpkg"):
        file.to_file(local_temporary_file, driver="GPKG")
    elif output_file.endswith(".csv"):
        file.to_csv(local_temporary_file, index=False)
    elif output_file.endswith(".geoparquet"):
        file.to_parquet(local_temporary_file, index=False)
    else:
        raise ValueError("Unsupported file format. Only .gpkg, .geoparquet and .csv are supported.")

    # Upload to S3
    output_path = S3Path(remote_path)
    output_path.upload_from(local_temporary_file)

    # Delete the local file after upload
    if os.path.exists(local_temporary_file):
        os.remove(local_temporary_file)



In [5]:
city_name = 'Medellin'
search_area_path = f"{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet"
search_area_file = gpd.GeoDataFrame(geometry=gpd.GeoSeries(selected_cities.loc[city_name]['merged_geometry']))

In [7]:
s3_save(file=search_area_file, 
        output_file=f"{city_name}_search_buffer.geoparquet", 
        output_temp_path='.', 
        remote_path=f"{SEARCH_BUFFER_PATH}/{city_name}")

In [10]:

for city_name in selected_cities.index:
    print(city_name)
    try:
        search_area_path = f"{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet"
        search_area_file = gpd.GeoDataFrame(geometry=gpd.GeoSeries(selected_cities.loc[city_name]['merged_geometry']))
        s3_save(file=search_area_file, 
            output_file=f"{city_name}_search_buffer.geoparquet", 
            output_temp_path='.', 
            remote_path=f"{SEARCH_BUFFER_PATH}/{city_name}")
    except:
        continue

Abeokuta
Abidjan
Abuja
Accra
Addis_Ababa
Aden
Agra
Agri
Ahmadabad
Aleppo
Alexandria
Algiers
Aligarh
Alor_Star
Amman
Amravati
Ankara
Ankara
Annaba
Anqing
Antalya
Antananarivo
Anyang
Arak
Arusha
Asuncion
Bacolod
Bac_Lieu
Baghdad
Bicheng
Baku
Bamako
Bamenda
Bandar_Lampung
Bandung
Bangkok
Bangui
Banjarbaru
Bankura
Barquisimeto
Barranquilla
Barreiras
Barretos
Bauchi
Beijing
Beira
Beirut
Belem
Belo_Horizonte
Benxi
Bhimavaram
Bhopal
Bilaspur
Bilbeis
Bogota
Bogor
Bouake
Bou_Saada
Buenos_Aires
Bukavu
Bukhara
Buraydah
Burewala
Bursa
Busan
Cabimas
Cabo_Frio
Cairo
Cali
Campinas
Can_Tho
Changzhou
Cape_Town
Caracas
Cartagena
Casablanca
Cebu_City
Changzhi
Chengde
Chengdu
Chennai
Chenzhou
Chittoor
Chongqing
Cirebon
Cochabamba
Coimbatore
Conakry
Constantine
Cordoba
Culiacan
Curico
Curitiba
Dakar
Dalat
Daloa
Damascus
Dar_es_Salaam
David
Denpasar
Dezhou
Dhaka
Dinajpur
Diyarbakir
Djibouti
Doha
Douala
Dubai
Durban
Enugu
Erbil
Eskisehir
Faisalabad
Feira_De_Santana
Fez
Florianopolis
Fortaleza
Freetown
Fuzhou

In [13]:

for city_name in selected_cities.index:
    print(city_name)
    try:
        search_area_path = f"{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet"
        search_area_file = gpd.GeoDataFrame(geometry=gpd.GeoSeries(selected_cities.loc[city_name]['merged_geometry']))
        s3_save(file=search_area_file, 
            output_file=f"{city_name}_search_buffer.geoparquet", 
            output_temp_path='.', 
            remote_path=f"{SEARCH_BUFFER_PATH}/{city_name}")
    except:
        continue

array(['Abeokuta', 'Abidjan', 'Abuja', 'Accra', 'Addis_Ababa', 'Aden',
       'Agra', 'Agri', 'Ahmadabad', 'Aleppo', 'Alexandria', 'Algiers',
       'Aligarh', 'Alor_Star', 'Amman', 'Amravati', 'Ankara', 'Ankara',
       'Annaba', 'Anqing', 'Antalya', 'Antananarivo', 'Anyang', 'Arak',
       'Arusha', 'Asuncion', 'Bacolod', 'Bac_Lieu', 'Baghdad', 'Bicheng',
       'Baku', 'Bamako', 'Bamenda', 'Bandar_Lampung', 'Bandung',
       'Bangkok', 'Bangui', 'Banjarbaru', 'Bankura', 'Barquisimeto',
       'Barranquilla', 'Barreiras', 'Barretos', 'Bauchi', 'Beijing',
       'Beira', 'Beirut', 'Belem', 'Belo_Horizonte', 'Benxi',
       'Bhimavaram', 'Bhopal', 'Bilaspur', 'Bilbeis', 'Bogota', 'Bogor',
       'Bouake', 'Bou_Saada', 'Buenos_Aires', 'Bukavu', 'Bukhara',
       'Buraydah', 'Burewala', 'Bursa', 'Busan', 'Cabimas', 'Cabo_Frio',
       'Cairo', 'Cali', 'Campinas', 'Can_Tho', 'Changzhou', 'Cape_Town',
       'Caracas', 'Cartagena', 'Casablanca', 'Cebu_City', 'Changzhi',
       'Chengde', '

In [17]:
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

# Copy files from SEARCH_BUFFER_PATH to GRIDS_PATH
for file_path in search_buffer_files[:10]:
    # Ensure we are not copying directories
    dest_path = GRIDS_PATH
    fs.copy(file_path, GRIDS_PATH)
    print(f"Copied {file_path} to {dest_path}")

print("All files copied successfully.")

Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abeokuta to s3://wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/grids
Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abidjan to s3://wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/grids
Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abuja to s3://wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/grids
Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Accra to s3://wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/grids
Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Addis_Ababa to s3://wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/grids
Copied wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Aden to s

In [12]:
search_buffer_files 

['wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abeokuta',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abidjan',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Abuja',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Accra',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Addis_Ababa',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Aden',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Agra',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Agri',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Ahmadabad',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/input/city_info/search_buffers/Aleppo',
 'wri-cities-sandbox/identifyingLandSubdivisions/data/inp